In [1]:
import psycopg2
import pandas as pd

In [10]:
# create psycopg2 connection
conn = psycopg2.connect(dbname='tpwd-tests',
                       user='postgres', host='localhost', port='5432',
                       password='pi3wqMa67*$GgX')
cur = conn.cursor()

In [13]:
search_term = 'apis'
query = """
    SELECT * FROM gbif_inverts_backbone
    WHERE canonicalname ~* ('\\y' || %(search_term)s) --select all results that contain keyword at the front of a word
    ORDER BY
        CASE
            WHEN canonicalname ~* ('^' || %(search_term)s) THEN 1 --prioritize results with term at front of entire string
            WHEN canonicalname ~* ('\\y' || %(search_term)s) THEN 2 --then results with term at front of non-first words
            ELSE 3
        END,
        canonicalname --sort each group alphabetically
    LIMIT 20
    """
cur.execute(query, {'search_term': search_term})


In [14]:
results = cur.fetchall()
print(list(results))

[('1334757', '7ddf754f-d193-4cc9-b351-99906754a03b', '4334.0', None, None, 'Apis Linnaeus, 1758', 'Linnaeus, 1758', 'Apis', 'Apis', None, None, 'genus', None, 'Linnæus, Carolus. 1758. Systema naturae per regna tria naturae, secundum classes, ordines, genera, species, cum characteribus, differentiis, synonymis, locis. Laurentii Salvii, Holmiae. Vol. Tomus I, Editio decima, reformata: i-ii, 1-824.', 'accepted', None, None, 'Animalia', 'Arthropoda', 'Insecta', 'Hymenoptera', 'Apidae', 'Apis'), ('12354605', 'da38f103-4410-43d1-b716-ea6b1b92bbac', '4334.0', None, None, 'Apis Fabricius', 'Fabricius', 'Apis', 'Apis', None, None, 'genus', None, None, 'doubtful', None, None, 'Animalia', 'Arthropoda', 'Insecta', 'Hymenoptera', 'Apidae', 'Apis'), ('7799978', '80b4b440-eaca-4860-aadf-d0dfdd3e856e', '1.0', None, None, 'Apis', None, 'Apis', 'Apis', None, None, 'genus', None, None, 'doubtful', None, None, 'Animalia', None, None, None, None, 'Apis'), ('9003872', 'c8227bb4-4143-443f-8cb2-51f9576aff14',

In [15]:
df = pd.DataFrame(results)
print(df)

          0                                     1          2          3   \
0    1334757  7ddf754f-d193-4cc9-b351-99906754a03b     4334.0       None   
1   12354605  da38f103-4410-43d1-b716-ea6b1b92bbac     4334.0       None   
2    7799978  80b4b440-eaca-4860-aadf-d0dfdd3e856e        1.0       None   
3    9003872  c8227bb4-4143-443f-8cb2-51f9576aff14  1334757.0       None   
4   11029307  7ddf754f-d193-4cc9-b351-99906754a03b  1339261.0  1339280.0   
5   10921250  7ddf754f-d193-4cc9-b351-99906754a03b  1339261.0  1339280.0   
6    9199522  7ddf754f-d193-4cc9-b351-99906754a03b  1340278.0  1340422.0   
7    6137054  7ddf754f-d193-4cc9-b351-99906754a03b  1339775.0  1340006.0   
8   12063421  7ddf754f-d193-4cc9-b351-99906754a03b  1338333.0  6096962.0   
9    9294043  c33ce2f2-c3cc-43a5-a380-fe4526d63650  1336587.0       None   
10  11034629  7ddf754f-d193-4cc9-b351-99906754a03b  1334757.0  1341976.0   
11   8816212  7ddf754f-d193-4cc9-b351-99906754a03b  1336906.0  1337092.0   
12  10948788

<h3>Here's a good starting point for the search!</h3>

In [ ]:
SELECT canonicalname, 
	CASE
		WHEN canonicalname ~* '^apis' THEN 1
		WHEN canonicalname ~* '\yapis\y' THEN 2
		WHEN canonicalname ~* '\yapis' AND canonicalname !~* '^apis' AND canonicalname !~ '\yapis\y' THEN 3
		ELSE 4
	END AS category
FROM gbif_inverts_backbone
	WHERE canonicalname ~* '\mapis'
	ORDER BY
	    CASE
			WHEN canonicalname ~* '^apis' THEN 1
			WHEN canonicalname ~* '\yapis\y' THEN 2
			WHEN canonicalname ~* '\yapis' AND canonicalname !~* '^apis' AND canonicalname !~ '\yapis\y' THEN 3
			ELSE 4
	    END,
		canonicalname